In [2]:
from jupyter_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output

import base64 #used to place a static image on the page
import numpy as np
import pandas as pd
from pymongo import MongoClient

from CRUD import AnimalShelter #import unique CRUD module

###########################
# Data Manipulation / Model
###########################

#hardcoded aacuser login information
username = "aacuser"
password = "AACUSERLOGIN"
shelter = AnimalShelter(username, password)

#Starter dataframe which includes all dogs within the different types of rescue
#Limited search to within the bounds of the filters to limit excessive data and improve load times
df = pd.DataFrame.from_records(shelter.read_no_id({'animal_type':'Dog', 
                                                    'breed':{'$in': ['Doberman Pinscher',
                                                                    'German Shepherd',
                                                                    'Golden Retriever',
                                                                    'Bloodhound',
                                                                    'Rottweiler',
                                                                    'Labrador Retriever Mix',
                                                                    'Chesapeake Bay Retriever',
                                                                    'Newfoundland',
                                                                    'Alaskan Malamute',
                                                                    'Old English Sheepdog',
                                                                    'Siberian Husky']},
                                                          '$or': [{'sex_upon_outcome': 'Intact Female'},
                                                                 {'sex_upon_outcome': 'Intact Male'}],
                                                          'age_upon_outcome_in_weeks':{'$gte': 20, '$lte': 300}}))

#Prepare static logo file
image_file = 'Grazioso_Salvare_Logo.png'
encoded_image = base64.b64encode(open(image_file, 'rb').read())

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Project Two')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div([
         html.A([
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), #Logo image import
                alt='test',
                    style={'height':'150px', 'width':'150px',
                          'marginLeft':'auto', 
                          'marginRight':'auto', 
                          'marginTop':'auto', 
                          'marginBottom':'auto'}),
        ], 
            href='https://www.snhu.edu', #connection to snhu website
           target='_blank'), #open link in new tab
        html.Center(html.B(html.H1('Animal Shelter Dashboard')), style={'marginLeft':'auto', 
                                                                       'marginRight':'auto', 
                                                                       'marginTop':'auto', 
                                                                       'marginBottom':'auto'}),

        #unique identifier
       html.Strong(html.P('Developed for the greater good by Joshua Love', style={'font-size':'12px',
                                                                                 'paddingTop':'120px'})),
    ],
    style={'display':'flex'}),
    html.Hr(),
    html.Div([
        
        #Button represents the option to reset filters
        html.Button('Reset Filters', id='reset-button', n_clicks=0, style={'float':'left', 'position':'relative'}),
        
        dcc.Dropdown( #Dropdown presents filter options to the user
            id='dropdown',
            options=[ #All currently available options
                {'label':'Water Rescue', 'value':'Water'},
                {'label':'Mountain or Wilderness Rescue', 'value':'Mountain'},
                {'label':'Disaster or Individual Tracking', 'value':'Disaster'},
            ],
            style={ #Keep dropdown on the leftside of the container, next to the button
                'width':'60%',
                'float':'left'
            },
            placeholder='Filter by rescue animal category'
        ),
    ], style={'display':'flex'}),
    
    #Interactive datatable element
    dash_table.DataTable(
        id='datatable-interactivity',
        
        columns=[ #Grab column names from dataframe
            {"name": i, "id": i} for i in df.columns
        ],
        data=df.to_dict('records'), #Put dataframe into dictionary
        sort_action='native', #allow sorting by column
        page_size=10, #10 entries per page
        style_cell={
            'textAlign':'left',
            'width':'auto',
            'whiteSpace':'normal',
            'height':'auto'
        },
        style_header={
            'backgroundColor':'rgb(230, 230, 230)',
            'fontWeight':'Bold',
        },

    ),
    html.Br(),
    html.Hr(),
    html.Br(),
    #Div which contains the map and graph side by side
    html.Div(className='row',
        style={'display' : 'flex', 'float':'left', 'paddingBottom':'20px'},
             children=[
                html.Div( #Graph
                id='graph-id',
                className='col s12 m6',
                ),
                html.Div( #Map
                id='map-id',
                className='col s12 m6',
                )
            ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it

#filter callback for both the dropdown and the button
@app.callback(
    [Output('datatable-interactivity', "data"), Output('graph-id', "children")], #returns df to datatable and dcc.Graph to graph div
    [Input('dropdown','value'), Input('reset-button', 'n_clicks')]) #takes the dropdown selection and button as input

def on_filter(value, btn):
    if value is None: #default value for initial loading
        #Start with initial dataframe which is within bounds of all available rescue dogs
        df = pd.DataFrame.from_records(shelter.read_no_id({'animal_type':'Dog', 
                                                    'breed':{'$in': ['Doberman Pinscher',
                                                                    'German Shepherd',
                                                                    'Golden Retriever',
                                                                    'Bloodhound',
                                                                    'Rottweiler',
                                                                    'Labrador Retriever Mix',
                                                                    'Chesapeake Bay Retriever',
                                                                    'Newfoundland',
                                                                    'Alaskan Malamute',
                                                                    'Old English Sheepdog',
                                                                    'Siberian Husky']},
                                                          '$or': [{'sex_upon_outcome': 'Intact Female'},
                                                                 {'sex_upon_outcome': 'Intact Male'}],
                                                          'age_upon_outcome_in_weeks':{'$gte': 20, '$lte': 300}}))
        
    elif value == 'Water': #Search for water rescue
        #pymongo query using CRUD module
        df = pd.DataFrame.from_records(shelter.read_no_id({'animal_type':'Dog',
                                                           'breed':{'$in': ['Labrador Retriever Mix',
                                                                         'Chesapeake Bay Retriever',
                                                                         'Newfoundland']},
                                                          'sex_upon_outcome':'Intact Female',
                                                          'age_upon_outcome_in_weeks':{'$gte':26, '$lte': 156}}))
        
    elif value == 'Mountain': #Search for mountain or wilderness rescue
        #pymongo query using CRUD module
        df = pd.DataFrame.from_records(shelter.read_no_id({'animal_type':'Dog',
                                                          'breed':{'$in': ['German Shepherd',
                                                                          'Alaskan Malamute',
                                                                          'Old English Sheepdog',
                                                                          'Siberian Husky',
                                                                          'Rottweiler']},
                                                          'sex_upon_outcome':'Intact Male',
                                                          'age_upon_outcome_in_weeks':{'$gte': 26, '$lte':156}}))
        
    elif value == 'Disaster': #Search for disaster or individual tracking
        #pymongo query using CRUD module
        df = pd.DataFrame.from_records(shelter.read_no_id({'animal_type':'Dog',
                                                          'breed':{'$in': ['Doberman Pinscher',
                                                                          'German Shepherd',
                                                                          'Golden Retriever',
                                                                          'Bloodhound',
                                                                          'Rottweiler']},
                                                          'sex_upon_outcome':'Intact Male',
                                                          'age_upon_outcome_in_weeks':{'$gte':20, '$lte':300}}))
    
    #Check if button has been pressed since last callback
    clicked = [p['prop_id'] for p in dash.callback_context.triggered][0]
    
    if 'reset-button' in clicked: #Button press is the reason for the callback
        #Return the initial dataframe to the datatable
        df = pd.DataFrame.from_records(shelter.read_no_id({'animal_type':'Dog', 
                                                    'breed':{'$in': ['Doberman Pinscher',
                                                                    'German Shepherd',
                                                                    'Golden Retriever',
                                                                    'Bloodhound',
                                                                    'Rottweiler',
                                                                    'Labrador Retriever Mix',
                                                                    'Chesapeake Bay Retriever',
                                                                    'Newfoundland',
                                                                    'Alaskan Malamute',
                                                                    'Old English Sheepdog',
                                                                    'Siberian Husky']},
                                                          '$or': [{'sex_upon_outcome': 'Intact Female'},
                                                                 {'sex_upon_outcome': 'Intact Male'}],
                                                          'age_upon_outcome_in_weeks':{'$gte': 20, '$lte': 300}}))
        
    """Start graph portion of the callback
        It's required to be in this section so that it's dynamic and
        responds to filter changes along with the table"""
    
    #Gather the counts for distinct breed values
    new_df = df['breed'].value_counts()
    
    #Create a new dataframe using the index and values from value_counts
    complete_df = pd.DataFrame({'breed':new_df.index, 'Count':new_df.values})
    
    #Create a new plotly express pie figure using the 
    fig = px.pie(complete_df, names=complete_df['breed'], values=complete_df['Count'], 
                 title='Distribution graph of current filter selection')
    
    #Place text of each pie slice within the slice and hide if too small
    fig.update_traces(textposition='inside')
    fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')

    #Returns the selected dataframe to the datatable as well as the constructed figure to the graph
    return df.to_dict('records'), dcc.Graph(figure=fig)

#Map callback
@app.callback(
    Output('map-id', "children"), #Outputs a dl.Map to the div element
    [Input('datatable-interactivity', "data")]) #Takes the current dictionary dataframe filter as an input
def update_map(viewData):
    
    #Convert dictionary back to dataframe
    dff = pd.DataFrame.from_dict(viewData)
    
    #Add the animals names within the dataframe to a numpy array
    animal_name = dff['name'].to_numpy()
    animal_breed = dff['breed'].to_numpy()
    
    #remove any values where lat or long are null - throws an error when trying to display
    animal_lat_clean = [x for x in dff['location_lat'] if str(x) != 'nan']
    animal_long_clean = [x for x in dff['location_long'] if str(x) != 'nan']
    
    #Array to store markers
    markers=[]
    
    #Loop to create markers
    for i in range(len(animal_lat_clean)):
        markers += [
            dl.Marker(
                position=(animal_lat_clean[i], animal_long_clean[i]), #Grab position for each unique animal
                children=[
                    dl.Tooltip(animal_breed[i]),
                    dl.Popup(animal_name[i] if animal_name[i] else "Unnamed") #Ensures every marker has a 'name' value
                ]
            )
        ]
    
    #Create cluster group
    cluster = dl.MarkerClusterGroup(children=markers)
    return [
        #return dash-leaflet map with cluster data centered at the average lat/long of current filter
        dl.Map(style={'width':'1000px', 'height':'500px'}, center=[(np.amax(animal_lat_clean)+np.amin(animal_lat_clean))*0.5,
                                                                   (np.amax(animal_long_clean)+np.amin(animal_long_clean))*0.5], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            cluster #Add cluster of markers to map
        ], preferCanvas=True) #Tell Dash to draw markers on canvas, slight performance improvement for this many markers
    ]

app.run_server()

Dash app running on http://127.0.0.1:8050/
